In [2]:
import os
import warnings
import re

import math
import sympy
import scipy
import cobra
import csv
import numpy as np
import pandas as pd
from micom import Community
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion, 
                                 double_gene_deletion, double_reaction_deletion)
from concurrent.futures import ThreadPoolExecutor, as_completed



In [3]:
file = open('Double_Lethals_Communities_FINAL.csv', 'w')
single_lethals_com = pd.read_csv('Single_Lethals_Communities_FINAL.csv')

/var/folders/dn/p3xbs4r94jqf_5q7_q53zj_c0000gn/T/ipykernel_41593/3397481871.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251

In [4]:
def process_pair(pair):

    orgs = pair.split(',')
    file_1 = f'Models/{orgs[0]}.xml'
    file_2 = f'Models/{orgs[1]}.xml'

    model_1 = read_sbml_model(file_1)
    model_2 = read_sbml_model(file_2)

    table = {'id': ['model_1', 'model_2'], 'file': [file_1, file_2]}
    data = pd.DataFrame(table)
    taxonomy = data

    com = Community(taxonomy, progress=False)
    com.medium = minimal_medium(com, 1)
    sol_WT = com.cooperative_tradeoff(fraction=0.7, fluxes=False)
    sol_df = sol_WT.members

    # Collect basic info
    row = [
        orgs[0], orgs[1],
        len(com.reactions), len(com.metabolites),
        sol_WT.growth_rate,
        sol_df._get_value('model_1', 'growth_rate'),
        sol_df._get_value('model_2', 'growth_rate'),
        sol_df._get_value('medium', 'reactions')
    ]

    # Perform double reaction deletion
    single_lethal_reactions_com = single_lethals_com[pair].dropna()
    print('Total : ', len(com.reactions))
    print('Single Lethals: ', len(single_lethal_reactions_com))
    
    double_lethal_reactions_com = []
    
    com_rxns_filtered = []
    for rxn in com.reactions:
        rxn_id = rxn.id
        if rxn_id not in single_lethal_reactions_com and rxn_id not in list(com.medium.keys()):
            com_rxns_filtered.append(rxn_id)
        
        
    com_rxn_pairs_filtered = []
    
    for i in range(len(com_rxns_filtered)):
        for j in range(i+1, len(com_rxns_filtered)):
            com_rxn_pairs_filtered.append([com_rxns_filtered[i], com_rxns_filtered[j]])
            
    print('Filtered : ', len(com_rxns_filtered))
    
    for rxn_pair in com_rxn_pairs_filtered:
        com_del = com
        with com_del:
            com_del.reactions.get_by_id(rxn_pair[0]).knock_out()
            com_del.reactions.get_by_id(rxn_pair[1]).knock_out()
            try:
                sol_del = com_del.cooperative_tradeoff(fraction = 0.7, fluxes=False)
                sol_del_df = sol_del.members

                if sol_del.growth_rate <= 0.1 * sol_WT.growth_rate or sol_del_df._get_value('model_1', 'growth_rate') <= sol_del.growth_rate * 0.1 or  sol_del_df._get_value('model_2', 'growth_rate') <= sol_del.growth_rate * 0.1:
                    double_lethal_reactions_com.append(rxn_pair)

            except ValueError:
                print('Error', rxn_pair)
                pass
    print(pair)
    print(len(double_lethal_reactions_com))
    file.write(pair, double_lethal_reactions_com)
    return row, pair, double_lethal_reactions_com




In [5]:
org_list_file = open('Models.csv')
org_names = [(i.split())[0] for i in org_list_file.readlines()]

pairwise_orgs = []
for i in range(len(org_names)):
    for j in range(i + 1, len(org_names)):
        pairwise_orgs.append(org_names[i] + ',' + org_names[j])

In [ ]:
process_pair(pairwise_orgs[0])

[20:37:36] WARNING  Could not identify an external compartment by name and choosing one with   ]8;id=694482;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py\boundary_types.py]8;;\:]8;id=565402;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py#90\90]8;;\
                    the most boundary reactions. That might be complete nonsense or change                         
                    suddenly. Consider renaming your compartments using `Model.compartments`                       
                    to fix this.                                                                                   

           WARNING  Could not identify an external compartment by name and choosing one with   ]8;id=167359;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py\boundary_types.py]8;;\:]8;id=315171;file:///Users/ashnanarain/anaconda3/envs/cobraenv/lib/python3.11/site-packages/cobra/medium/boundary_types.py#90\90]8;;\
                    the most boundary reactions. That might be complete nonsense or change                         
                    suddenly. Consider renaming your compartments using `Model.compartments`                       
                    to fix this.                                                                                   

Total :  2635
Single Lethals:  675
Filtered :  2585
Error ['26DAPLLAT__model_1', 'EX_pi(e)__model_2']
Error ['2MBCOATA__model_1', 'EX_gncore2(e)__model_2']
Error ['2MBCOATA__model_1', 'PItex__model_2']
Error ['3HAD10M12__model_1', 'DM_kdo2lipid4L(c)__model_2']
Error ['3HAD11M12__model_1', 'DM_kdo2lipid4L(c)__model_2']
Error ['3HAD11M12__model_1', 'SUCCt2rpp__model_2']
Error ['3HAD12M14__model_1', 'AICART__model_2']
Error ['3HAD13M14__model_1', 'SUCCtex__model_1']
Error ['3HAD13M14__model_1', 'AICART__model_2']
Error ['3HAD14M16__model_1', 'AICART__model_2']
Error ['3HAD14M16__model_1', 'FUM__model_2']
Error ['3HAD14M16__model_1', 'MDH__model_2']
Error ['3HAD14M16__model_1', 'PItex__model_2']
Error ['3HAD15M16__model_1', 'EX_pi(e)__model_2']
Error ['3HAD15M16__model_1', 'FUM__model_2']
Error ['3HAD15M16__model_1', 'PItex__model_2']
Error ['3HAD4M6__model_1', 'ADSL2r__model_2']
Error ['3HAD4M6__model_1', 'PItex__model_2']
Error ['3HAD5M6__model_1', 'ADSL2r__model_2']
Error ['3HAD5M6__mod